# Capture the Flag (RL - Policy Gradient)

- Seung Hyun Kim
- skim449@illinois.edu

## Implementation Details

- Actor-critic experience buffer.
- Off-policy: Training is done at the end of the rollout

### Sampling
- [ ] Mini-batch to update 'average' gradient
- [x] Experience Replay for Random Sampling
- [ ] Importance Sampling
    
### Deterministic Policy Gradient
- [ ] DDPG
- [ ] MADDPG

### Stability and Reducing Variance
- [x] Gradient clipping
- [x] Normalized Reward/Advantage
- [ ] Target Network
- [ ] TRPO
- [ ] PPO

### Multiprocessing
- [ ] Synchronous Environment Rolling
- [ ] Synchronous Training (A2C)
- [ ] Asynchronous Training (A3C)

### Applied Training Methods:
- [ ] Self-play
- [ ] Batch Policy

## Notes

- This notebook includes:
    - Building the structure of policy driven network.
    - Training with/without render
    - Saver that save model and weights to ./model directory
    - Writer that will record some necessary datas to ./logs

- This notebook does not include:
    - Simulation with RL policy
        - The simulation can be done using policy_RL.py
    - cap_test.py is changed appropriately.
    
## References :
- https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb (source)
- https://www.youtube.com/watch?v=PDbXPBwOavc
- https://github.com/lilianweng/deep-reinforcement-learning-gym/blob/master/playground/policies/actor_critic.py (source)
- https://github.com/spro/practical-pytorch/blob/master/reinforce-gridworld/reinforce-gridworld.ipynb

## TODO:

- enemy with different policies (zero, patrol)
- stochastic interaction
- Reward -> only 100 for completion (with small observation)

In [1]:
!rm -rf logs/B4R4_Rzero_AC_MonteCarlo/ model/B4R4_Rzero_AC_MonteCarlo

In [2]:
TRAIN_NAME='B4R4_Rzero_AC_MonteCarlo'
LOG_PATH='./logs/'+TRAIN_NAME
MODEL_PATH='./model/' + TRAIN_NAME
GPU_CAPACITY=0.2 # gpu capacity in percentage (adjustable)

In [3]:
import os

import signal

import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.layers as layers
from tensorflow.python.client import device_lib
import matplotlib.pyplot as plt
%matplotlib inline

import time
import gym
import gym_cap
import gym_cap.envs.const as CONST
import numpy as np
import random

# the modules that you can use to generate the policy. 
import policy.random
import policy.roomba
import policy.policy_RL
import policy.zeros

# Data Processing Module
from utility.dataModule import one_hot_encoder
from utility.utils import MovingAverage as MA
from utility.utils import Experience_buffer, discount_rewards

%load_ext autoreload
%autoreload 2

## Hyperparameters

In [4]:
# Replay Variables
total_episodes=5000000
max_ep = 150
update_frequency = 4
batch_size = 512
experience_size=2048

# Saving Related
save_network_frequency = 1000
save_stat_frequency = 100
moving_average_step = 100

# Training Variables
LEARNING_RATE_FIX = True
LEARNINGRATE_AC  = 1e-4
LEARNINGRATE_CRITIC = 3e-4
LR_DECAY = 0.9995
LR_CRITIC_DECAY = 0.995
LR_FINAL = 1e-4
LR_CRITIC_FINAL = 5e-4
gamma = 0.99
discount_factor = 0.99

# Env Settings
MAP_SIZE = 20
VISION_RANGE = 9 # What decide the network size !!!
VISION_dX, VISION_dY = 2*VISION_RANGE+1, 2*VISION_RANGE+1

## Environment Setting

In [ ]:
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
#Create a directory to save episode playback gifs to
if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

In [ ]:
env = gym.make("cap-v0") # initialize the environment
#plt.imshow(env.render(mode='rgb_array'))
policy_red = policy.zeros.PolicyGen(env.get_map, env.get_team_red)
action_space = 5
n_agent = len(env.get_team_blue)
print('red number : ', len(env.get_team_red))
print('blue number : ', len(env.get_team_blue))

red number :  6
blue number :  6


## Policy Network

In [ ]:
#Used to initialize weights for policy and value output layers
def normalized_columns_initializer(std=1.0):
    def _initializer(shape, dtype=None, partition_info=None):
        out = np.random.randn(*shape).astype(np.float32)
        out *= std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
        return tf.constant(out)
    return _initializer

In [ ]:
class Agent():
    def __init__(self, in_size, action_size, grad_clip_norm):
        # Parameters
        self.grad_clip_norm = grad_clip_norm
        
        # Learning Rate Variables
        self.learning_rate = tf.placeholder(tf.float32, shape=None, name='learning_rate')
        self.learning_rate_critic = tf.placeholder(tf.float32, shape=None, name='learning_rate_critic')
        
        # Placeholders
        self.state_input = tf.placeholder(shape=in_size,dtype=tf.float32, name='state')
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        self.action_OH = tf.one_hot(self.action_holder, action_size)
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32, name='reward')
        self.td_target_holder = tf.placeholder(shape=[None], dtype=tf.float32, name='td_target')
        self.advantage_holder = tf.placeholder(shape=[None], dtype=tf.float32, name='adv')
        
        
        # Feed-Forward Network
        # Actor stream
        layer = slim.conv2d(self.state_input, 32, [5,5], activation_fn=tf.nn.relu,
                            weights_initializer=layers.xavier_initializer_conv2d(),
                            biases_initializer=tf.zeros_initializer(),
                            padding='VALID',
                            scope='conv1')
        layer = slim.max_pool2d(layer, [2,2])
        layer = slim.conv2d(layer, 64, [3,3], activation_fn=tf.nn.relu,
                            weights_initializer=layers.xavier_initializer_conv2d(),
                            biases_initializer=tf.zeros_initializer(),
                            padding='VALID',
                            scope='conv2')
        layer = slim.max_pool2d(layer, [2,2])
        layer = slim.conv2d(layer, 64, [2,2], activation_fn=tf.nn.relu,
                            weights_initializer=layers.xavier_initializer_conv2d(),
                            biases_initializer=tf.zeros_initializer(),
                            padding='VALID',
                            scope='conv3')
        layer = slim.flatten(layer)
        actor = layers.fully_connected(layer, 128,
                                    weights_initializer=normalized_columns_initializer(0.001),
                                    activation_fn=tf.nn.relu,
                                    scope='dense_1')
        self.actor = layers.fully_connected(actor, action_size,
                                    weights_initializer=normalized_columns_initializer(0.001),
                                    activation_fn=None,
                                    scope='dense_2')
        self.output = tf.nn.softmax(self.actor, name='action')
        self.output_argmax = tf.argmax(self.output, axis=1,output_type=tf.int32, name='argmax')
        
        # Value stream
        self.critic = layers.fully_connected(layer, 1,
                                             weights_initializer=normalized_columns_initializer(1.0),
                                             activation_fn=None,
                                             scope='critic_1')
        self.critic = tf.reshape(self.critic, [-1])
        
        # Feed Backward
        # - compute the  loss, and use it to find gradient, and update the network
        # - May be need to add bootstrap value at the end of the value
        #self.selected=tf.equal(self.action_holder, self.output_argmax)
        with tf.name_scope('train'):
            self.entropy = -tf.reduce_sum(self.output * tf.log(self.output), name='entropy')
            #self.advantage = self.td_target_holder - self.critic
            '''mean,std = tf.nn.moments(self.advantage, [-1])
            self.norm_td_err = (self.advantage - mean) / (std + 1e-8) # Normalize td error'''

            # loss critic
            self.loss_critic = tf.reduce_sum(tf.square(self.td_target_holder - self.critic))

            # loss actor
            self.policy_outputs = tf.reduce_sum(self.output * self.action_OH, 1)
            self.objective_function = tf.log(self.policy_outputs)
            self.loss_actor = -tf.reduce_sum(self.objective_function*self.advantage_holder) # using holder, no need to stop gradient
            
        with tf.name_scope('update'):
            self.loss = 0.5*self.loss_critic + self.loss_actor# - self.entropy * 0.0001
            self.optimizer = tf.train.AdamOptimizer(self.learning_rate)
            self.grads = self.optimizer.compute_gradients(self.loss)
            if self.grad_clip_norm:
                self.grads = [(tf.clip_by_norm(grad, self.grad_clip_norm), var) for grad, var in self.grads]
            self.update_batch = self.optimizer.apply_gradients(self.grads)
            self.extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            
        # Summary
        # Histogram output
        with tf.name_scope('debug_parameters'):
            tf.summary.histogram('output', self.output)
            tf.summary.histogram('actor', self.actor)
            tf.summary.histogram('critic', self.critic)        
            tf.summary.histogram('action', self.action_holder)
            tf.summary.histogram('objective_function', self.objective_function)
            tf.summary.histogram('td_target', self.td_target_holder)
            tf.summary.histogram('rewards_in', self.reward_holder)
            tf.summary.histogram('advantage_in', self.advantage_holder)
            
        with tf.name_scope('gradient'):
            for grad, var in self.grads:
                tf.summary.histogram(var.name+'_grad', grad)
            
        
        # Graph summary Loss
        with tf.name_scope('summary'):
            tf.summary.scalar(name='actor_loss', tensor=self.loss_actor)
            tf.summary.scalar(name='critic_loss', tensor=self.loss_critic)
            tf.summary.scalar(name='total_loss', tensor=self.loss)
            tf.summary.scalar(name='Entropy', tensor=self.entropy)
        
        with tf.name_scope('weights_bias'):
            # Histogram weights and bias
            for var in slim.get_model_variables():
                tf.summary.histogram(var.op.name, var)
                
        with tf.name_scope('Learning_Rate'):
            # Learning Rate
            tf.summary.scalar(name='actor_lr', tensor=self.learning_rate)

In [ ]:
tf.reset_default_graph() # Clear the Tensorflow graph.
myAgent = Agent(in_size=[None,VISION_dX,VISION_dY,6],action_size=5, grad_clip_norm=50) #Load the agent.
with tf.variable_scope('global_step'):
    global_step = tf.Variable(0, trainable=False, name='global_step') # global step
    increment_global_step_op = tf.assign(global_step, global_step+1)
merged = tf.summary.merge_all()

INFO:tensorflow:Summary name conv1/weights:0_grad is illegal; using conv1/weights_0_grad instead.
INFO:tensorflow:Summary name conv1/biases:0_grad is illegal; using conv1/biases_0_grad instead.
INFO:tensorflow:Summary name conv2/weights:0_grad is illegal; using conv2/weights_0_grad instead.
INFO:tensorflow:Summary name conv2/biases:0_grad is illegal; using conv2/biases_0_grad instead.
INFO:tensorflow:Summary name conv3/weights:0_grad is illegal; using conv3/weights_0_grad instead.
INFO:tensorflow:Summary name conv3/biases:0_grad is illegal; using conv3/biases_0_grad instead.
INFO:tensorflow:Summary name dense_1/weights:0_grad is illegal; using dense_1/weights_0_grad instead.
INFO:tensorflow:Summary name dense_1/biases:0_grad is illegal; using dense_1/biases_0_grad instead.
INFO:tensorflow:Summary name dense_2/weights:0_grad is illegal; using dense_2/weights_0_grad instead.
INFO:tensorflow:Summary name dense_2/biases:0_grad is illegal; using dense_2/biases_0_grad instead.
INFO:tensorflo

## Session

In [ ]:
# Launch the session
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_CAPACITY, allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#sess = tf.Session()

ma_reward = MA(moving_average_step)
ma_length = MA(moving_average_step)
ma_captured = MA(moving_average_step)

# Setup Save and Restore Network
saver = tf.train.Saver(tf.global_variables())
writer = tf.summary.FileWriter(LOG_PATH, sess.graph)

ckpt = tf.train.get_checkpoint_state(MODEL_PATH)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Load Model : ", ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    print("Initialized Variables")

Initialized Variables


In [ ]:
def record(summary_):
    with tf.device('/cpu:0'): 
        summary = tf.Summary()
        summary.value.add(tag='Records/mean_reward', simple_value=ma_reward())
        summary.value.add(tag='Records/mean_length', simple_value=ma_length())
        summary.value.add(tag='Records/mean_succeed', simple_value=ma_captured())
        writer.add_summary(summary, sess.run(global_step))
        
        writer.add_summary(summary_, sess.run(global_step))
        
        writer.flush()

In [ ]:
#Close session
def handler(signum, frame):
    print('Reset Taking Too Long')
    raise Exception('Action took too much time')

In [ ]:
def policy_rollout():
    # Run single episode, return the results
    
    # Temporary fix for episode reset
    flag = True
    while flag:
        signal.signal(signal.SIGALRM, handler)
        signal.alarm(3) #Set the parameter to the amount of seconds you want to wait
        try:
            s = env.reset(map_size=MAP_SIZE, policy_red=policy_red)
            flag = False
        except:
            print('timeout. retry:')
            flag = True
        signal.alarm(0) #Disables the alarm
    

    #obs = one_hot_encoder(s, env.get_team_blue) # partial observation
    obs_next = one_hot_encoder(env._env, env.get_team_blue, VISION_RANGE)

    
    ep_history = []
    indv_history = [[] for _ in range(len(env.get_team_blue))]
    was_alive = [ag.isAlive for ag in env.get_team_blue]
    
    prev_reward=0
    total_reward = 0
    frame=0
    
    for frame in range(max_ep+1):
        obs = obs_next
        
        with tf.device('/cpu:0'):
            act_prob, v0 = sess.run([myAgent.output, myAgent.critic], feed_dict={myAgent.state_input:obs})
        act = [np.random.choice(action_space, p=act_prob[x]/sum(act_prob[x])) for x in range(n_agent)] # divide by sum : normalize
        behavior_policy = act_prob#np.log(act_chosen)
        
        s,r1,d,_ = env.step(act) #Get our reward for taking an action given a bandit.
        r = r1-prev_reward
        #if r >=1 and r < 100: # ignore capturing
        #    r = 0
        if frame == max_ep and d == False:
            r = -100
            r1 = -100
            d = True
        total_reward += r
        
        if d:
            v1 = np.array([0.0 for _ in range(n_agent)])
        else:
            obs_next = one_hot_encoder(env._env, env.get_team_blue, VISION_RANGE) # Full Observation
            v1 = sess.run(myAgent.critic, feed_dict={myAgent.state_input:obs_next})
        
        # Push history for individual that 'was' alive previous frame
        # [state, action, reward, td-target, td-error(advantage), behavior_policy]
        for idx, agent in enumerate(env.get_team_blue):
            if was_alive[idx]:
                indv_history[idx].append([obs[idx],act[idx],r,gamma*v1[idx],gamma*v1[idx]-v0[idx],behavior_policy[idx]])
      
        # State Transition
        prev_reward = r1
        was_alive = [ag.isAlive for ag in env.get_team_blue]
        
        if d:
            break

    # Policy rollout for all agents are done.
    # Calculate Advantage for individual histories
    for idx, history in enumerate(indv_history):
        if len(history)==0:
            continue
        _history = np.array(history)
        _history[:,2] = discount_rewards(_history[:,2], discount_factor) / 100.0#, normalize=True)
        _history[:,3] += _history[:,2]
        _history[:,4] += _history[:,2]
        ep_history.extend(_history)
      
    if len(ep_history) > 0:        
        ep_history = np.stack(ep_history)

    return [frame, ep_history, r1, env.blue_win, total_reward]


## Training

In [ ]:
if __name__=='__main__':
    ep = sess.run(global_step)
    exp_buffer = Experience_buffer(experience_shape=6, buffer_size=experience_size)
    batch_history = []
    progbar = tf.keras.utils.Progbar(total_episodes,width=5, interval=0.5)
    try:
        while ep < total_episodes+1:
            progbar.update(ep) # update progress bar
            
            # Run episode
            frame, history, reward, did_won, total_reward = policy_rollout()

            # Add history
            exp_buffer.add(history)

            if ep % update_frequency == 0 and ep != 0:
                batch_history = exp_buffer.sample(batch_size)
                feed_dict={myAgent.learning_rate          :LEARNINGRATE_AC,
                           myAgent.state_input            :np.stack(batch_history[:,0]),
                           myAgent.action_holder          :batch_history[:,1],
                           myAgent.reward_holder          :batch_history[:,2],
                           myAgent.td_target_holder       :batch_history[:,3],
                           myAgent.advantage_holder       :batch_history[:,4]}
                with tf.device('/gpu:0'):
                    sess.run([myAgent.update_batch, myAgent.extra_update_ops], feed_dict=feed_dict)    
                exp_buffer.flush()
            
            # decay lr
            if not LEARNING_RATE_FIX:
                LEARNINGRATE_AC  = max(LEARNINGRATE_AC*LR_DECAY,LR_FINAL)
                LEARNINGRATE_CRITIC = max(LEARNINGRATE_CRITIC*LR_CRITIC_DECAY,LR_CRITIC_FINAL)
            
            # summarize and record
            ma_reward.append(reward)
            ma_length.append(frame)
            ma_captured.append(env.blue_win)   
            
            if ep % save_stat_frequency == 0 and ep != 0 and len(batch_history) > 0:
                summary_ = sess.run(merged, feed_dict=feed_dict)
                record(summary_)

            # save weight
            if ep % save_network_frequency == 0 and ep != 0:
                saver.save(sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_step)

            # Proceed to next episode
            ep += 1
            
            sess.run(increment_global_step_op)

    except KeyboardInterrupt:
        print('\n\nManually stopped the training (KeyboardInterrupt)');
        saver.save(sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_step)
        print("save: ", sess.run(global_step), 'episodes')

    301/5000000 [.....] - ETA: 1839:39:51